<a href="https://colab.research.google.com/github/SaiVarad1/Meditation_App_NLP_Analysis/blob/main/LIS501_Meditation_Poster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# libraries
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas as pd
from matplotlib import pyplot as plt
from math import ceil
import numpy as np
from google.colab import files

!pip install bertopic[spacy]
#!pip install bertopic
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')





Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 71.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 103.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 72.2 MB/s eta 0:00:00
  Preparing metad

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
#Read In CSV
uploaded=files.upload()
df = pd.read_csv('appstore_calm_us.csv')

Saving appstore_calm_us.csv to appstore_calm_us.csv


In [ ]:
# Split data into positive and negative reviews
positive_reviews = df[df['rating'] > 3]['review']
negative_reviews = df[df['rating'] < 3]['review']

positive_reviews

0      I updated the calm app and just out of curiosi...
3      I absolutely love love love this app!!!!♥️ onc...
4      Honestly, I had come to a point in my life whe...
5      I love this app and use it all the time. Loads...
6      This app has had an incredible impact on my me...
                             ...                        
994    It used to be amazing. Then you locked the 7 D...
995    I want a refund for my subscription. I paid fo...
996    I was diagnosed with severe anxiety a few year...
997    Honestly I had never really meditated before b...
999    Overall, this app is pretty amazing. I suffer ...
Name: review, Length: 766, dtype: object

In [ ]:

stop_words = nltk.corpus.stopwords.words('english')

wtk = nltk.tokenize.RegexpTokenizer(r'\w+')
wnl = nltk.stem.wordnet.WordNetLemmatizer()

def normalize_corpus(articles):
    norm_Articles = []
    for article in articles:
        article = article.lower()
        article_tokens = [token.strip() for token in wtk.tokenize(article)] #Return a copy of the string S with leading and trailing whitespace removed.
        article_tokens = [wnl.lemmatize(token) for token in article_tokens if not token.isnumeric()] #lemmatize strings numbers
        article_tokens = [token for token in article_tokens if len(token) > 1] #remove single character tokens
        article_tokens = [token for token in article_tokens if token not in stop_words] #remove stop words
        article_tokens = list(filter(None, article_tokens))
        if article_tokens:
            norm_Articles.append(article_tokens)
    return norm_Articles

norm_positive_reviews = normalize_corpus(positive_reviews)
norm_negative_reviews=normalize_corpus(negative_reviews)

list_norm_positive_reviews= [' '.join(lst) for lst in norm_positive_reviews]
list_norm_negative_reviews=[' '.join(lst) for lst in norm_negative_reviews]

In [ ]:
docs_positive=list_norm_positive_reviews
docs_negative=list_norm_negative_reviews

In [ ]:
from sentence_transformers import SentenceTransformer, util

model=SentenceTransformer('paraphrase-MiniLM-L3-v2')

In [ ]:
#creating a function

def Bertopic(doc,n_neigbors_param=5, n_components_param=5, top_n_words_param=15):
  # Step 1 - Extract embeddings
  #embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
  embedding_model=model

  # Step 2 - Reduce dimensionality
  umap_model = UMAP(n_neighbors=n_neigbors_param, n_components=n_components_param, min_dist=0.0, metric='cosine')

  # Step 3 - Cluster reduced embeddings
  hdbscan_model = HDBSCAN(min_cluster_size=3, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

  # Step 4 - Tokenize topics
  vectorizer_model = CountVectorizer(stop_words="english")

  # Step 5 - Create topic representation
  ctfidf_model = ClassTfidfTransformer()

  # Step 6 - (Optional) Fine-tune topic representations with
  # a `bertopic.representation` model
  representation_model = KeyBERTInspired()

  # All steps together renamed in a new model with these params
  Better_BERTopic_model = BERTopic(
  top_n_words=top_n_words_param,
  embedding_model=embedding_model,          # Step 1 - Extract embeddings
  umap_model=umap_model,                    # Step 2 - Reduce dimensionality
  hdbscan_model=hdbscan_model,              # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
  representation_model=representation_model # Step 6 - (Optional) Fine-tune topic represenations
  )

  BetterBA_topics, BetterBA_probs = Better_BERTopic_model.fit_transform(doc)
  return Better_BERTopic_model.visualize_topics()


In [ ]:
Bertopic(docs_positive)

In [ ]:
Bertopic(docs_negative)


In [ ]:
Bertopic(docs_positive,n_components_param=10,n_neigbors_param=15,top_n_words_param=30)

In [ ]:
Bertopic(docs_negative,n_components_param=8,n_neigbors_param=15,top_n_words_param=30)

In [ ]:
# third alteration to parameters:
#<top_n_words,  <n_neigbors(gives more global view), <components
#decrease all 3 params

Bertopic(docs_positive,n_components_param=3,n_neigbors_param=5,top_n_words_param=5)

In [ ]:
Bertopic(docs_negative,n_components_param=3,n_neigbors_param=5,top_n_words_param=5)

In [ ]:
#Conclusion
#We can see that increasing n_components too much leads to BERtopic having a hard time clustering the high-dimensional embeddings, while lowering the value too much causes not sufficient info for proper clusters

# top_n_words:  # We can see that increasing n_words too much creates less coherent topic formation, while having a value that's too low causes less informative/representative topics


 # Increasing n_neigbors creates a more global view of the embedded structure and larger clusters being created, while decreasing it leads to a more local view and fewer clusters.